# Cascade decay of a Ne 1s-hole: Simulate the final ion distribution

# **This nootebook is still under construction !!!**

In [1]:
using JAC

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5401211853016568705\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6630359014716717077\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5401211853016568705\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


The computation of the cascade tree alone does not provide much information which could be compared with experiment. To facilitate the prediction of cascades or the comparison with experiment, the amplitudes from the various decay pathways need to be combined properly in order to predict different spectra and observables. Such observables are

(1)
(2)

What does it include: Let us have a look at the definition of such a `Cascade.Simulation`:


In [2]:
? Cascade.Simulation

`struct  Cascade.Simulation`  ... defines a simulation on some given cascade (data).

```
+ properties      ::Array{Cascade.Property,1}   ... Properties that are considered in this simulation of the cascade (data).
+ method          ::Cascade.SimulationMethod    ... Method that is used in the cascade simulation; cf. Cascade.SimulationMethod.
+ settings        ::Cascade.SimulationSettings  ... Settings for performing these simulations.
```


What is seen ... and which information needed to be provided for such a computation